In [18]:
# Set-Ups 
from flask import Flask, request, jsonify, send_file, render_template
from werkzeug.utils import secure_filename
import joblib
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, inspect, select, text, and_, or_, func
from sqlalchemy import Table, MetaData

from datetime import datetime

import json

In [3]:
# Connect to database
passwd = 'FyZSMAiH'
w_engine = create_engine('mysql+pymysql://e0376935:' + passwd + '@localhost:3306/olist', echo=True)

m2 = MetaData(bind=w_engine)
m2.reflect()

orders = Table("orders", m2, autoload=True)

2022-03-23 07:04:00,708 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-23 07:04:00,708 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 07:04:00,724 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-03-23 07:04:00,724 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 07:04:00,732 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-03-23 07:04:00,732 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 07:04:00,754 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `olist`
2022-03-23 07:04:00,755 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 07:04:00,772 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `customers`
2022-03-23 07:04:00,773 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 07:04:00,783 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `geo_avg`
2022-03-23 07:04:00,783 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 07:04:00,792 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `geo_loc`
2022-03-23 07:04:00,793 INFO sqlalch

Order Filter Function

In [16]:
# Try the query
stmt1 = select([orders]).\
        where(orders.c.order_status == "delivered").\
        order_by(orders.c.purchase_time).limit(10)
    
with w_engine.connect() as con:
        d1 = con.execute(stmt1)
    
d1.fetchall()

2022-03-23 07:13:04,703 INFO sqlalchemy.engine.Engine SELECT orders.order_id, orders.customer_id, orders.order_status, orders.purchase_time, orders.approval_time, orders.delivered_carrier, orders.delivered_customer, orders.estimated_delivery 
FROM orders 
WHERE orders.order_status = %(order_status_1)s ORDER BY orders.purchase_time 
 LIMIT %(param_1)s
2022-03-23 07:13:04,703 INFO sqlalchemy.engine.Engine [cached since 542s ago] {'order_status_1': 'delivered', 'param_1': 10}


[('bfbd0f9bdef84302105ad712db648a6c', '86dc2ffce2dfff336de2f386a786e574', 'delivered', datetime.datetime(2016, 9, 15, 12, 16, 38), datetime.datetime(2016, 9, 15, 12, 16, 38), datetime.datetime(2016, 11, 7, 17, 11, 53), datetime.datetime(2016, 11, 9, 7, 47, 38), datetime.datetime(2016, 10, 4, 0, 0)),
 ('3b697a20d9e427646d92567910af6d57', '355077684019f7f60a031656bd7262b8', 'delivered', datetime.datetime(2016, 10, 3, 9, 44, 50), datetime.datetime(2016, 10, 6, 15, 50, 54), datetime.datetime(2016, 10, 23, 14, 2, 13), datetime.datetime(2016, 10, 26, 14, 2, 13), datetime.datetime(2016, 10, 27, 0, 0)),
 ('be5bc2f0da14d8071e2d45451ad119d9', '7ec40b22510fdbea1b08921dd39e63d8', 'delivered', datetime.datetime(2016, 10, 3, 16, 56, 50), datetime.datetime(2016, 10, 6, 16, 3, 44), datetime.datetime(2016, 10, 21, 16, 33, 46), datetime.datetime(2016, 10, 27, 18, 19, 38), datetime.datetime(2016, 11, 7, 0, 0)),
 ('a41c8759fbe7aab36ea07e038b2d4465', '6f989332712d3222b6571b1cf5b835ce', 'delivered', datetim

In [5]:
datetime(2016, 10, 24, 16, 33, 45) > datetime.strptime('2017-06-26 21:14:55', '%Y-%m-%d %H:%M:%S')

False

In [9]:
# Careful! The time input must in this format: '2017-02-28'
# If we want to include time, like "2017-02-28 00:00:00", use '%Y-%m-%d %H:%M:%S' instead

def select_orders(start_time, end_time, status):
    start_datetime = datetime.strptime(start_time, '%Y-%m-%d')
    end_datetime = datetime.strptime(end_time, '%Y-%m-%d')

    query1 = select([orders]).\
        where(orders.c.order_status == status).\
        where(orders.c.purchase_time > start_datetime).\
        where(orders.c.purchase_time < end_datetime).\
        order_by(orders.c.purchase_time).limit(10)
    
    with w_engine.connect() as con:
        d1 = con.execute(query1)
    
    return d1.fetchall()

In [15]:
s1 = select_orders('2017-02-28','2017-03-15')
type(s1) # s1 is a list

2022-03-23 07:09:24,416 INFO sqlalchemy.engine.Engine SELECT orders.order_id, orders.customer_id, orders.order_status, orders.purchase_time, orders.approval_time, orders.delivered_carrier, orders.delivered_customer, orders.estimated_delivery 
FROM orders 
WHERE orders.order_status = %(order_status_1)s AND orders.purchase_time > %(purchase_time_1)s AND orders.purchase_time < %(purchase_time_2)s ORDER BY orders.purchase_time 
 LIMIT %(param_1)s
2022-03-23 07:09:24,416 INFO sqlalchemy.engine.Engine [cached since 315.3s ago] {'order_status_1': 'delivered', 'purchase_time_1': datetime.datetime(2017, 2, 28, 0, 0), 'purchase_time_2': datetime.datetime(2017, 3, 15, 0, 0), 'param_1': 10}


'7b7ce86399877a0908f5af4041da975d'

In [17]:
app = Flask(__name__)

# Order Filter
# This API should return a list containing orders from one time to another
# "status" argument determines the order is "delievered" ot "cancelled" etc.
@app.route("/orders", methods=["GET"])
def order_filter():
    start_time = request.args.get('start_time')
    end_time = request.args.get('end_time')
    status = request.args.get('status')

    return json.dump(select_orders(start_time, end_time, status))

In [32]:
type(s1[1])
len(s1)
s1[0][7]

datetime.datetime(2017, 4, 3, 0, 0)

In [30]:
d1 = []
for i in range(len(s1)):
    d1.append(s1[i][0])

type(d1)

list